# Part 1: Generating data with `FHI-aims` (supporting notebook)

In [ ]:
from os.path import exists, join
import chemiscope
 
from rholearn.utils import system
from rholearn.utils.io import unpickle_dict
from rholearn.options import get_options

dft_options = get_options("dft", "doslearn")
dft_options

In [ ]:
# Display the nuclear geometries with chemiscope
frames = system.read_frames_from_xyz(dft_options["XYZ"])
frame_idxs = range(len(frames))
chemiscope.show(system.chemfiles_frame_to_ase_frame(frames), mode="structure")

## 1.2: Converge SCF

In [6]:
# Check all SCF calculations have converged
scf_dir = lambda A: f"data/raw/{A}"

not_conv = []
for A in frame_idxs:
    path = join(scf_dir(A), "aims.out")
    if not exists(path):
        not_conv.append(A)
    else:
        with open(path, "r") as f:
            if "Have a nice day." not in f.read():
                not_conv.append(A)

assert len(not_conv) == 0, f"SCF not converged for {not_conv}"

In [7]:
# Confirm again all SCF calculations have converged via the parsed aims.out files
for A in frame_idxs:
    calc_info = unpickle_dict(join(scf_dir(A), "calc_info.pickle"))
    assert calc_info["scf"]["converged"]